In [27]:
# Import necessary libraries
from selenium import webdriver  # for web scraping using a browser
from bs4 import BeautifulSoup  # for parsing HTML content
import pandas as pd  # for data manipulation and handling
import time  # for adding delays in code execution
from tqdm import tqdm  # for displaying progress bars

In [28]:
# Initialize a Chrome WebDriver instance
browser = webdriver.Chrome()

In [29]:
# Prompt the user to input a YouTube channel name
channelName = input("Please enter a YouTube channel: ")

In [30]:
# Construct a Google search link for the input channel name
link = 'https://www.google.com/search?q=' + channelName + ' youtube'

In [32]:
# Open the constructed link in the browser
browser.get(link)

In [33]:
# Parse the HTML content of the page
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [34]:
# Extract the link of the first search result (which is assumed to be the channel link)
for i in soup.find_all('div', class_='MjjYud'):
    link = i.find('a').get('href')
    break

In [35]:
# Open the channel's videos page in the browser
browser.get(link + '/' + 'videos')

In [36]:
# Parse the HTML content of the page
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [37]:
# Scroll down the page to load more videos dynamically
for i in tqdm(range(0, 5000, 1000)):
    browser.execute_script('window.scrollTo(0,' + str(i) + ')')
    time.sleep(0.1)

100%|██████████| 5/5 [00:00<00:00,  8.35it/s]


In [38]:
# Parse the HTML content of the page after scrolling
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [39]:
# Find all video items on the page
video_items = soup.find_all('ytd-rich-item-renderer')

In [42]:
# Iterate over each video item and extract relevant information
data = []
for item in video_items:
    link = "https://www.youtube.com/" + item.find('a', class_='yt-simple-endpoint focus-on-expand style-scope ytd-rich-grid-media').get('href')
    title = item.find('a', class_='yt-simple-endpoint focus-on-expand style-scope ytd-rich-grid-media').get('title')
    views = (item.find('span', class_="inline-metadata-item style-scope ytd-video-meta-block").text.split(" ")[0])
    upload_time = (item.find_all('span', class_="inline-metadata-item style-scope ytd-video-meta-block")[1].text)
    data.append([link, title, views, upload_time])

In [43]:
# Save the DataFrame to a CSV file with the channel name as filename
df=pd.DataFrame(data,columns=['Link','Title','Views','UploadTime'])

df.to_csv(channelName + '.csv', index=False)